In [78]:
import gzip as gz
import os
from io import StringIO
import pandas as pd
import datetime as DT
import numpy as np
import itertools

from bokeh.io import show, output_notebook
from bokeh.models import FactorRange
from bokeh.plotting import figure

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV, GridSearchCV

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

output_notebook()

import warnings
warnings.filterwarnings('ignore')

Loading BokehJS ...

# Introduction

We put ourselves in the position of a manager in charge of a statewide campaign. As part of voter outreach, state and national campaigns typically include targeted mailings. Let's say that we have the resources to send 100,000 such mailings. Who should we send them to?

In [2]:
#data_path = "Data"
#if not os.path.isdir(data_path):
#    os.mkdir(data_path)
#if len(os.listdir(data_path)) == 0:
#    !wget -O /Data/1.gz https://www6.sos.state.oh.us/ords/f?p=VOTERFTP:DOWNLOAD::FILE:NO:2:P2_PRODUCT_NUMBER:363
#    !wget -O /Data/2.gz https://www6.sos.state.oh.us/ords/f?p=VOTERFTP:DOWNLOAD::FILE:NO:2:P2_PRODUCT_NUMBER:364
#    !wget -O /Data/3.gz https://www6.sos.state.oh.us/ords/f?p=VOTERFTP:DOWNLOAD::FILE:NO:2:P2_PRODUCT_NUMBER:365
#    !wget -O /Data/4.gz https://www6.sos.state.oh.us/ords/f?p=VOTERFTP:DOWNLOAD::FILE:NO:2:P2_PRODUCT_NUMBER:366

The Ohio voter file (https://www6.sos.state.oh.us/ords/f?p=VOTERFTP:STWD:::#stwdVtrFiles) contains a wealth of information about registered voters. We will use that data to make a prediction model to aid in our voter outreach. The data set contains over 8 million entries and over 100 columns, so it takes some time to load.

# Data preparation

In [3]:
data_path = "data"
files = [os.path.join(data_path, file) for file in os.listdir(data_path) 
         if os.path.isfile(os.path.join(data_path, file)) and file.endswith(".gz")]

In [4]:
csv_files = []
for file in files:
    with gz.open(file, "r") as z:
        file_content = z.read().decode("utf-8")
        csv_files.append(file_content)

In [5]:
df_list = []
for csv_file in csv_files:
    df = pd.read_csv(StringIO(csv_file), index_col=None, header=0)
    df_list.append(df)
    
df = pd.concat(df_list, axis = 0, ignore_index = True)

In [6]:
df_reduced = df.iloc[:, [1, 3 , 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 31, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104]]
df_reduced.head()

,COUNTY_NUMBER,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,DATE_OF_BIRTH,REGISTRATION_DATE,VOTER_STATUS,PARTY_AFFILIATION,RESIDENTIAL_ADDRESS1,...,PRIMARY-03/15/2016,GENERAL-06/07/2016,PRIMARY-09/13/2016,GENERAL-11/08/2016,PRIMARY-05/02/2017,PRIMARY-09/12/2017,GENERAL-11/07/2017,PRIMARY-05/08/2018,GENERAL-08/07/2018,GENERAL-11/06/2018
0,17,MOLLENCOPF,OLIVIA,ANITA,NaN,1942-02-17,1973-08-13,ACTIVE,R,6612 WINDFALL RD,...,R,NaN,NaN,X,NaN,NaN,X,NaN,NaN,X
1,17,KELLER,TAMMERA,ANNETTE,NaN,1967-04-22,2018-10-09,ACTIVE,R,1140 WESTMOOR DR,...,R,NaN,NaN,X,NaN,NaN,X,NaN,NaN,NaN
2,18,GILLISPIE,MARTHA,H,NaN,1995-10-02,2018-10-09,ACTIVE,D,1673 CUMBERLAND RD,...,D,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN,X
3,12,PARRILL,PHYLLIS,ANN,NaN,1934-05-01,1955-09-28,ACTIVE,D,2150 STOWE DR,...,D,NaN,NaN,X,X,NaN,X,D,NaN,X
4,8,LAKES,BILLICE,L,NaN,1936-08-26,1997-08-19,ACTIVE,D,5280 US HIGHWAY 62 AND 68,...,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
now = pd.Timestamp(DT.datetime.now())
df_reduced['DATE_OF_BIRTH'] = pd.to_datetime(df['DATE_OF_BIRTH'])    # 1
df_reduced['DATE_OF_BIRTH'] = df_reduced['DATE_OF_BIRTH'].where(df_reduced['DATE_OF_BIRTH'] < now, df_reduced['DATE_OF_BIRTH'] -  np.timedelta64(100, 'Y'))   # 2
df_reduced['AGE'] = (now - df_reduced['DATE_OF_BIRTH']).astype('<m8[Y]')
df_reduced = df_reduced[df_reduced['RESIDENTIAL_ZIP'].notnull()]
df_reduced['RESIDENTIAL_ZIP'] = df_reduced['RESIDENTIAL_ZIP'].astype(np.int)

#Development only
df_reduced = df_reduced.sample(n=1000000, replace=False)

We add the average income in a voter's zip code as a feature. The average income data is derived from the following data set: https://www.irs.gov/statistics/soi-tax-stats-individual-income-tax-statistics-2016-zip-code-data-soi. While the median income would be a better statistic, it is unfortunately not readily available for the recent years.

In [8]:
income_df = pd.read_csv(f"{data_path}/16zpallnoagi.csv", encoding = "ISO-8859-1")
income_df.index = income_df['ZIPCODE']
income_df['AVG_INCOME'] = income_df['A00200'] * 1000 / income_df['N00200']
avg_income = income_df['AVG_INCOME']
avg_income.head()

ZIPCODE
0        47775.961436
35004    51611.648352
35005    37054.511278
35006    42025.961538
35007    53577.406680
Name: AVG_INCOME, dtype: float64

In [9]:
df_reduced = df_reduced[df_reduced["RESIDENTIAL_ZIP"].isin(avg_income.index)]
df_reduced["AVG_INCOME"] = df_reduced["RESIDENTIAL_ZIP"].apply(lambda x: avg_income[x])

The population density is certain to be an important feature in predicting someone's political views, as rural residents are in general more conservative and more likely to vote Republican. The population density by zip code data was obtained here: https://blog.splitwise.com/2014/01/06/free-us-population-density-and-unemployment-rate-by-zip-code/.

In [10]:
pop_density = pd.read_csv(f"{data_path}/Zipcode-ZCTA-Population-Density-And-Area-Unsorted.csv", encoding = "ISO-8859-1")
pop_density['Zip/ZCTA'] = pop_density['Zip/ZCTA'].astype(np.int)
pop_density.index = pop_density['Zip/ZCTA']
pop_density = pop_density['Density Per Sq Mile']
pop_density[pop_density > 0].head()

Zip/ZCTA
1001     1465.565461
1002      527.751031
1003    14587.904360
1005      114.800416
1007      278.270615
Name: Density Per Sq Mile, dtype: float64

In [11]:
df_reduced = df_reduced[df_reduced["RESIDENTIAL_ZIP"].isin(pop_density.index)]
df_reduced["POP_DENSITY"] = df_reduced["RESIDENTIAL_ZIP"].apply(lambda x: pop_density[x])

In [56]:
voted_primary = df_reduced[df_reduced['PRIMARY-05/08/2018'].notnull() 
                           & df_reduced['PRIMARY-05/08/2018'].apply(lambda x: x == 'R' or x == 'D')]
voted_primary['is_D'] = df_reduced['PRIMARY-05/08/2018'] == 'D'
voted_primary['is_R'] = df_reduced['PRIMARY-05/08/2018'] == 'R'

# A bit of exploratory analysis

In [13]:
counts = [voted_primary.is_D.sum(), voted_primary.is_R.sum()]
parties = ["Democrat", "Republican"]
colors = ["blue", "red"]

p = figure(x_range=parties, plot_height=350, title="Number of Politically Active Voters By Party", toolbar_location=None, tools="")
p.vbar(x=parties, top=counts, width=0.9, alpha=0.5, fill_color=colors)
p.xgrid.grid_line_color = None
p.y_range.start = 0

show(p)

In [14]:
def pairwise(iterable):
    a, b = itertools.tee(iterable)
    next(b, None)
    return zip(a, b)

income_levels = [i*10000 for i in range(2, 12)]
factors = list(itertools.chain(*[((f"{i}-{j}", "R"), (f"{i}-{j}", "D")) for i, j in pairwise(income_levels)]))
incomes_R = [voted_primary[voted_primary.is_R & (voted_primary["AVG_INCOME"] > i) & (voted_primary["AVG_INCOME"] < j)].shape[0] for i, j in pairwise(income_levels)]
incomes_D = [voted_primary[voted_primary.is_D & (voted_primary["AVG_INCOME"] > i) & (voted_primary["AVG_INCOME"] < j)].shape[0] for i, j in pairwise(income_levels)]
incomes = list(itertools.chain(*zip(incomes_R, incomes_D)))
colors = list(itertools.chain(*[("red", "blue") for i, j in pairwise(income_levels)]))

p = figure(x_range=FactorRange(*factors), plot_height=500, plot_width=1000, title="Incomes by Party", toolbar_location=None, tools="")
p.vbar(x=factors, top=incomes, width=0.9, alpha=0.5, color=colors)
p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xaxis.axis_label = "Incomes ($)"

show(p)

In [15]:
pop_min, pop_max = voted_primary["POP_DENSITY"].min(), voted_primary["POP_DENSITY"].max()
pop_levels = [0, 1000, 3000, np.inf] #np.linspace(pop_min, pop_max, 4)
pop_designations = ["rural", "suburban", "urban"]

factors = list(itertools.chain(*[((i, "R"), (i, "D")) for i in pop_designations]))
pop_R = [voted_primary[voted_primary.is_R & (voted_primary["POP_DENSITY"] > i) & (voted_primary["POP_DENSITY"] < j)].shape[0] for i, j in pairwise(pop_levels)]
pop_D = [voted_primary[voted_primary.is_D & (voted_primary["POP_DENSITY"] > i) & (voted_primary["POP_DENSITY"] < j)].shape[0] for i, j in pairwise(pop_levels)]
pops = list(itertools.chain(*zip(pop_R, pop_D)))
colors = list(itertools.chain(*[("red", "blue") for i, j in pairwise(pop_levels)]))

p = figure(x_range=FactorRange(*factors), plot_height=500, plot_width=1000, title="Population Level by Party", toolbar_location=None, tools="")
p.vbar(x=factors, top=pops, width=0.9, alpha=0.5, color=colors)
p.y_range.start = 0
p.x_range.range_padding = 0.1

show(p)

In [16]:
age_levels = [18, 25, 35, 45, 55, 65, 75, 85, 95, 100, 120]

factors = list(itertools.chain(*[((f"{i}-{j}", "R"), (f"{i}-{j}", "D")) for i, j in pairwise(age_levels)]))
age_R = [voted_primary[voted_primary.is_R & (voted_primary["AGE"] > i) & (voted_primary["AGE"] < j)].shape[0] for i, j in pairwise(age_levels)]
age_D = [voted_primary[voted_primary.is_D & (voted_primary["AGE"] > i) & (voted_primary["AGE"] < j)].shape[0] for i, j in pairwise(age_levels)]
ages = list(itertools.chain(*zip(age_R, age_D)))
colors = list(itertools.chain(*[("red", "blue") for i, j in pairwise(age_levels)]))

p = figure(x_range=FactorRange(*factors), plot_height=500, plot_width=1000, title="Age by Party", toolbar_location=None, tools="")
p.vbar(x=factors, top=ages, width=0.9, alpha=0.5, color=colors)
p.y_range.start = 0
p.x_range.range_padding = 0.1

show(p)

Let's calculate a couple more initeresting statistics from the 2018 midterm election:

In [17]:
primary_turnouts = [df_reduced.groupby(["PARTY_AFFILIATION"]).get_group(party)['PRIMARY-05/08/2018'].notnull().sum()
                        / (df_reduced["PARTY_AFFILIATION"] == party).sum() for party in ['D', 'R']]
parties = ["Democrat", "Republican"]
colors = ["blue", "red"]

p = figure(x_range=parties, plot_height=350, title="Primary Turnout by Party", toolbar_location=None, tools="")
p.vbar(x=parties, top=primary_turnouts, width=0.9, alpha=0.5, fill_color=colors)
p.xgrid.grid_line_color = None
p.y_range.start = 0

show(p)

In [18]:
general_turnouts = [df_reduced.groupby(["PARTY_AFFILIATION"]).get_group(party)['GENERAL-11/06/2018'].notnull().sum()
                        / (df_reduced["PARTY_AFFILIATION"] == party).sum() for party in ['D', 'R']]
parties = ["Democrat", "Republican"]
colors = ["blue", "red"]

p = figure(x_range=parties, plot_height=350, title="General Election Turnout by Party", toolbar_location=None, tools="")
p.vbar(x=parties, top=general_turnouts, width=0.9, alpha=0.5, fill_color=colors)
p.xgrid.grid_line_color = None
p.y_range.start = 0

show(p)

# Prediction model

In [34]:
features = ['AGE', 'AVG_INCOME', 'POP_DENSITY']
target = 'is_D'

model_df = voted_primary[features + ['is_R', 'is_D']].dropna().reset_index()
train_df, holdout_df, y_train, y_holdout = train_test_split(
    model_df[features], 
    model_df[target], test_size=0.1)

train_df[target] = y_train
holdout_df[target] = y_holdout

train_df.reset_index(inplace=True)
holdout_df.reset_index(inplace=True)

print(train_df.shape[0], train_df[target].mean())
print(holdout_df.shape[0], holdout_df[target].mean())

175152 0.45425116470265825
19462 0.4562223820778954


In [21]:
k_fold = KFold(n_splits=5)

In [22]:
def get_cv_results(classifier):
    
    results = []
    for train, test in k_fold.split(train_df):
        classifier.fit(train_df.loc[train, features], train_df.loc[train, target])
        y_predicted = classifier.predict(train_df.loc[test, features])
        accuracy = accuracy_score(train_df.loc[test, target], y_predicted)
        results.append(accuracy)
    
    return np.mean(results), np.std(results)

##### Logistic regression

Let's try to optimize the regularization strength hyperameter:

In [23]:
c = [0.001,0.01,0.1,1,10,100]
penalty = ['l1', 'l2']

grid = {'C': c,
           'penalty': penalty}

logreg = LogisticRegression()

logreg_random = GridSearchCV(estimator = logreg, param_grid = grid, cv = 5, verbose=2, n_jobs = -1)
logreg_random.fit(train_df.loc[:, features], train_df[target])
logreg_best_params = logreg_random.best_params_

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:   12.8s finished


In [24]:
logreg = LogisticRegression(**logreg_random.best_params_)

get_cv_results(logreg)

(0.6446457590073171, 0.003353367676066037)

##### Decision Tree

In [25]:
min_samples_split = [2, 5, 7, 10, 15, 20, 50, 60, 70, 80, 90, 100, 120, 150]
max_depth = [3, 4, 5, 6, 7, 8, 9, 10]

grid = {'max_depth': max_depth,
               'min_samples_split': min_samples_split}

dtree = DecisionTreeClassifier()

dtree_random = GridSearchCV(estimator = dtree, param_grid = grid, cv = 5, verbose=2, n_jobs = -1)
dtree_random.fit(train_df.loc[:, features], train_df[target])
dtree_best_params = dtree_random.best_params_

Fitting 5 folds for each of 112 candidates, totalling 560 fits


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:    8.3s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:   21.6s
[Parallel(n_jobs=-1)]: Done 560 out of 560 | elapsed:   35.1s finished


In [26]:
dtree = DecisionTreeClassifier(**dtree_random.best_params_)

get_cv_results(dtree)

(0.674157301709844, 0.0019122958581305946)

##### Random Forest

In [28]:
n_estimators = [int(x) for x in np.linspace(start = 50, stop = 2000, num = 10)]
max_depth = [3, 4, 5, 6, 7, 8, 9, 10]
max_features = ['auto', 'sqrt']
max_depth.append(None)
min_samples_split = [500, 750, 1000, 1250, 1500]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rforest = RandomForestClassifier()

rf_random = RandomizedSearchCV(estimator = rforest, param_distributions = random_grid, n_iter = 20, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(train_df.loc[:, features], train_df[target])
rf_best_params = rf_random.best_params_

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 14.7min finished


In [29]:
rforest = RandomForestClassifier(**rf_random.best_params_)

get_cv_results(rforest)

(0.6739860072388232, 0.002282073405253723)

##### Gradient Boosting

In [30]:
learning_rate = [1, 0.5, 0.25, 0.1, 0.05, 0.01]
n_estimators = [1, 2, 4, 8, 16, 32, 64, 100, 200]
max_depth = np.linspace(1, 32, 32, endpoint=True)
min_samples_split = np.linspace(0.1, 1.0, 10, endpoint=True)
min_samples_leaf = np.linspace(0.1, 0.5, 5, endpoint=True)
max_features = list(range(1,len(features)))

random_grid = {'learning_rate': learning_rate,
               'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'max_features': max_features}

gb = GradientBoostingClassifier()

gb_random = RandomizedSearchCV(estimator = gb, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
gb_random.fit(train_df.loc[:, features], train_df[target])
gb_best_params = gb_random.best_params_

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   22.3s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.8min finished


In [31]:
gbm = GradientBoostingClassifier(**gb_random.best_params_)

get_cv_results(gbm)

(0.676007093851453, 0.0030586389029867047)

Conclusion: 

All models produce similar accuracy, so we are going to choose the decision tree classiffier for our task of selecting 100,000 voters to whom we are going to send our targeted campaign mailings. The accuracy is around 68%, which is not terrible given the limited number of features and the assumptions that are being made.

##### Voter Selection

Before we use our classification model, we need to narrow our data to a pool of voters who are most likely to be swayed by our campaign letters. It makes no sense to target voters who are registered with the opposite party. Similarly, it makes little sense to target voters who are registered with our party and vote regularly, since their votes are most likely already secured. Therefore, we are going to include those voters who are not affiliated with either party and Democrats who have not voted for at least two election cycles.

In [64]:
target_to_party = {
    'is_D': 'D',
    'is_R': 'R'
}

not_recent_voter = df_reduced.loc[:, ['PRIMARY-03/15/2016', 'GENERAL-06/07/2016', 'PRIMARY-09/13/2016',
       'GENERAL-11/08/2016', 'PRIMARY-05/02/2017', 'PRIMARY-09/12/2017',
       'GENERAL-11/07/2017', 'PRIMARY-05/08/2018', 'GENERAL-08/07/2018',
       'GENERAL-11/06/2018']].notnull().sum(1) == 0
target_party = df_reduced['PARTY_AFFILIATION'] == target_to_party[target]
not_affiliated = df_reduced['PARTY_AFFILIATION'].isnull()

possible_choices = df_reduced[(target_party & not_recent_voter) | not_affiliated]
possible_choices.shape

(587814, 33)

In [65]:
classifier = DecisionTreeClassifier(**dtree_random.best_params_)
classifier.fit(voted_primary.loc[:, features], voted_primary.loc[:, target])
predicted = classifier.predict_proba(possible_choices.loc[:, features])
possible_choices[f"P({target_to_party[target]})"] = predicted[:,0]
possible_choices.head()

,COUNTY_NUMBER,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,DATE_OF_BIRTH,REGISTRATION_DATE,VOTER_STATUS,PARTY_AFFILIATION,RESIDENTIAL_ADDRESS1,...,GENERAL-11/07/2017,PRIMARY-05/08/2018,GENERAL-08/07/2018,GENERAL-11/06/2018,AGE,AVG_INCOME,POP_DENSITY,is_D,is_R,P(D)
5753894,51,KITTS,SHANE,E,NaN,1980-04-30,2008-02-04,CONFIRMATION,NaN,569 PARK ST,...,NaN,NaN,NaN,NaN,38.0,37818.641304,280.736812,False,False,0.632281
6949473,76,BEGGS,PETER,V,NaN,1983-08-10,2011-07-22,ACTIVE,NaN,236 19TH ST NW,...,NaN,NaN,NaN,NaN,35.0,41221.066667,3284.398888,False,False,0.471429
5715904,57,DUNN HARDIN,TODJ,D,NaN,1997-03-24,2015-09-08,ACTIVE,NaN,4423 BLUEBERRY AVE,...,NaN,NaN,NaN,NaN,21.0,26797.101449,4381.047581,False,False,0.129487
6975891,76,BURNS-ALLEN,SUMMER,ROSE,NaN,1979-08-08,2008-09-02,ACTIVE,NaN,2806 WOOD OWL ST NE,...,X,NaN,NaN,NaN,39.0,21890.977444,1574.142916,False,False,0.090000
2256491,41,LUKACENA,KAYLEE,MARIE,NaN,1993-09-22,2012-01-03,ACTIVE,NaN,1994 TOWNSHIP ROAD 246,...,NaN,NaN,NaN,NaN,25.0,40317.647059,200.839014,False,False,0.676245


In [73]:
selected = possible_choices.nlargest(100000, ["P(D)"])
selected.head()

,COUNTY_NUMBER,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,DATE_OF_BIRTH,REGISTRATION_DATE,VOTER_STATUS,PARTY_AFFILIATION,RESIDENTIAL_ADDRESS1,...,GENERAL-11/07/2017,PRIMARY-05/08/2018,GENERAL-08/07/2018,GENERAL-11/06/2018,AGE,AVG_INCOME,POP_DENSITY,is_D,is_R,P(D)
1851607,2,STAHLER,JEREMY,RYAN,NaN,1976-10-12,2016-11-08,ACTIVE,NaN,4390 FT AMANDA RD,...,X,NaN,NaN,X,42.0,45456.297919,1496.145667,False,False,1.0
2534859,25,THURSTON,MARLENE,J,NaN,1933-05-09,2008-04-09,CONFIRMATION,NaN,1526 BROWN RD,...,NaN,NaN,NaN,NaN,85.0,27958.078603,2719.737627,False,False,1.0
3899230,31,ANDERSON,ANDY,NaN,NaN,1966-11-29,2000-10-04,ACTIVE,NaN,502 HOOVEN RD,...,NaN,NaN,NaN,NaN,52.0,27808.333333,5411.764706,False,False,1.0
3832497,31,STORY,JEFFREY,ALAN,NaN,1954-07-16,1996-10-16,ACTIVE,NaN,404 OHIO AVE,...,NaN,NaN,NaN,X,64.0,27808.333333,5411.764706,False,False,1.0
2161857,2,MAUCH,MIKKI,KIM,NaN,1976-12-03,1995-03-29,ACTIVE,NaN,101 S DALE DR,...,X,NaN,NaN,NaN,42.0,45456.297919,1496.145667,False,False,1.0


In [76]:
hist, edges = np.histogram(selected["P(D)"], density=True, bins=10)

p = figure(title="Prediction Probabilities", tools="")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="white", alpha=0.5)
p.xaxis.axis_label = "Probability"
p.yaxis.visible = False
show(p)

Not bad - almost all of the selected voters have a prediction probability of over 75%. We can be confident that we are not wasting our resources by sending them letters.